
# 📚 Multi‑Book Chapter/Page NLP Pipeline (spaCy, Overlap, Correlation)

**End‑to‑end pipeline:**
- Walks your folder tree: `main_folder/{physics,chemistry,biology}_{chapters,pages}/*.md`
- Cleans text (lowercase, optional LaTeX removal)
- Tokenizes with **spaCy** (fast `nlp.pipe`), optional lemmatization
- Token statistics (with & without stopwords)
- Document–Term Matrix (DTM)
- Pairwise overlaps (intersection & **Jaccard** %) – intrabook & interbook
- Correlation matrices (Pearson) over term‑frequency vectors
- Saves CSVs + PNG heatmaps to an `outputs/` folder

> Put this notebook where `main_folder/` is accessible, or set `BASE_DIR` below.


## 1) Configuration

In [16]:
from pathlib import Path

# Auto-detect MAIN_FOLDER (fallback to main_folder)
_base_candidates = [Path("MAIN_FOLDER"), Path("main_folder")]
BASE_DIR = next((p for p in _base_candidates if p.exists()), _base_candidates[-1])
print("Using BASE_DIR:", BASE_DIR.resolve())

SUBJECTS = ["physics", "chemistry", "biology"]
MODES    = ["chapters", "pages"]  # keep both; you can set to ["chapters"] or ["pages"] to run one at a time

# Processing options (unchanged)
REMOVE_LATEX = True
KEEP_ALNUM   = False
LEMMATIZE    = True
INCLUDE_NUMS = True
MIN_TERM_LEN = 2
BATCH_SIZE   = 32    # NEW: spaCy pipe batch size for speed

STOPWORDS_EXTRA = set()

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True, parents=True)
print(f"Outputs -> {OUT_DIR.resolve()}")


Using BASE_DIR: /Users/tanmayagarwal/Desktop/My_Computer/Columbia/Fall_2025/RA_LLM/main_folder/main_folder
Outputs -> /Users/tanmayagarwal/Desktop/My_Computer/Columbia/Fall_2025/RA_LLM/main_folder/outputs


## 2) Install & Imports

In [17]:

# If running on Colab/fresh env, uncomment installs:
# !pip install -q spacy pandas numpy matplotlib tqdm
# !python -m spacy download en_core_web_sm

import re, json, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import spacy

try:
    nlp = spacy.load("en_core_web_sm", disable=["ner", "parser", "textcat"])
except OSError:
    # Uncomment to auto-download if you have internet:
    # !python -m spacy download en_core_web_sm
    nlp = spacy.blank("en")
    print("Warning: Using blank('en') model. Install 'en_core_web_sm' for lemmatization/stopwords.")

# spaCy stopwords
SPACY_STOPS = set(getattr(getattr(nlp, "Defaults", None), "stop_words", set()))
STOPWORDS = SPACY_STOPS | set(map(str.lower, STOPWORDS_EXTRA))
print(f"Loaded {len(SPACY_STOPS)} spaCy stopwords (+{len(STOPWORDS_EXTRA)} custom).")


Loaded 326 spaCy stopwords (+0 custom).



## 3) 30‑sec spaCy Primer
- **Tokenization**: splits text into tokens (words/punct)
- **Lemmatization**: base form (e.g., running → run)
- **Stopwords**: frequent low‑info words (e.g., the, and)
We'll output results **with** and **without** stopwords.


## 4) Helpers: Load, Clean, Tokenize

In [18]:
import re

LATEX_PATTERNS = [
    (r"\$\$.*?\$\$", re.DOTALL),   # $$ ... $$
    (r"\$.*?\$", re.DOTALL),       # $ ... $
    (r"\\\[.*?\\\]", re.DOTALL),   # \[ ... \]
    (r"\\\(.*?\\\)", re.DOTALL),   # \( ... \)
]

def strip_latex(text: str) -> str:
    t = text
    for pat, flags in LATEX_PATTERNS:
        t = re.sub(pat, " ", t, flags=flags)
    return t

ALNUM_RE = re.compile(r"[^a-z0-9\s]+")
WS_RE    = re.compile(r"\s+")

def basic_clean(text: str) -> str:
    t = text.lower()
    if REMOVE_LATEX:
        t = strip_latex(t)
    # strip markdown images/links/code
    t = re.sub(r"!\[[^\]]*\]\([^\)]*\)", " ", t)   # images
    t = re.sub(r"\[[^\]]*\]\([^\)]*\)", " ", t)    # links
    t = re.sub(r"`[^`]*`", " ", t)                 # inline code
    if KEEP_ALNUM:
        t = ALNUM_RE.sub(" ", t)
    t = WS_RE.sub(" ", t).strip()
    return t

def _numeric_sort_key(stem: str):
    # chapter_01 -> 1, page_0001 -> 1; falls back to name if no digits
    m = re.search(r"(\d+)$", stem)
    if m:
        return int(m.group(1))
    nums = re.findall(r"\d+", stem)
    return int(nums[-1]) if nums else stem

def iter_files(base: Path, subjects, modes):
    for subj in subjects:
        for mode in modes:
            folder = base / f"{subj}_{mode}"
            if not folder.exists():
                continue
            files = list(folder.glob("*.md"))
            files.sort(key=lambda p: _numeric_sort_key(p.stem))  # numeric order
            for p in files:
                yield subj, mode, p

def tokenize_doc(doc):
    toks_all, toks_no_stop = [], []
    for tok in doc:
        if tok.is_space or tok.is_punct:
            continue
        if not INCLUDE_NUMS and tok.like_num:
            continue
        term = (tok.lemma_ if (LEMMATIZE and getattr(tok, "lemma_", None)) else tok.text).lower()
        if len(term) < MIN_TERM_LEN:
            continue
        toks_all.append(term)
        # STOPWORDS is assembled in the Imports cell (spaCy + custom)
        if term not in STOPWORDS:
            toks_no_stop.append(term)
    return toks_all, toks_no_stop


## 5) Load & Process the Corpus (fast `nlp.pipe`)

In [19]:
# Discover files
records = []
for subj, mode, path in iter_files(BASE_DIR, SUBJECTS, MODES):
    records.append({"subject": subj, "mode": mode, "path": path, "name": f"{subj}_{mode}_{path.stem}"})
print(f"Found {len(records)} files.")
if not records:
    print("No markdown files found. Expected MAIN_FOLDER/*_{chapters|pages}/*.md")

# Clean texts
texts  = []
names  = []
for r in records:
    raw   = r["path"].read_text(encoding="utf-8", errors="ignore")
    clean = basic_clean(raw)
    texts.append(clean)
    names.append(r["name"])
print(f"Prepared {len(texts)} cleaned texts.")


Found 0 files.
No markdown files found. Expected MAIN_FOLDER/*_{chapters|pages}/*.md
Prepared 0 cleaned texts.


In [14]:
from tqdm import tqdm

tokens_all_map   = {}
tokens_nostop_map = {}

for name, doc in tqdm(zip(names, nlp.pipe(texts, batch_size=BATCH_SIZE)), total=len(names), desc="spaCy pipe"):
    toks_all, toks_nostop = tokenize_doc(doc)
    tokens_all_map[name]   = toks_all
    tokens_nostop_map[name] = toks_nostop

print("Docs processed:", len(tokens_all_map))


spaCy pipe: 0it [00:00, ?it/s]

Docs processed: 0


## 6) Token Statistics (with & without stopwords)

In [15]:

import pandas as pd

def token_stats(tokens_map: dict) -> pd.DataFrame:
    rows = []
    for name, toks in tokens_map.items():
        rows.append({
            "doc": name,
            "num_tokens": len(toks),
            "num_unique": len(set(toks)),
        })
    return pd.DataFrame(rows).set_index("doc").sort_index()

stats_all    = token_stats(tokens_all_map)
stats_nostop = token_stats(tokens_nostop_map)

stats_all.to_csv(OUT_DIR / "token_stats_with_stopwords.csv")
stats_nostop.to_csv(OUT_DIR / "token_stats_without_stopwords.csv")

display(stats_all.head())
display(stats_nostop.head())
print("Saved token stats CSVs.")


KeyError: "None of ['doc'] are in the columns"

## 7) Document–Term Matrix (DTM)

In [ ]:

from collections import Counter
import numpy as np

def build_dtm(tokens_map: dict) -> pd.DataFrame:
    vocab = set()
    counters = {}
    for name, toks in tokens_map.items():
        c = Counter(toks)
        counters[name] = c
        vocab |= set(c.keys())
    vocab = sorted(vocab)
    df = pd.DataFrame(index=sorted(tokens_map.keys()), columns=vocab, data=0, dtype=np.int32)
    for name, c in counters.items():
        for term, cnt in c.items():
            df.at[name, term] = cnt
    return df

dtm_all    = build_dtm(tokens_all_map)
dtm_nostop = build_dtm(tokens_nostop_map)

dtm_all.to_csv(OUT_DIR / "dtm_with_stopwords.csv")
dtm_nostop.to_csv(OUT_DIR / "dtm_without_stopwords.csv")
dtm_all.shape, dtm_nostop.shape


## 8) Pairwise Overlap (Intersection & Jaccard %)

In [ ]:

import numpy as np

def jaccard_matrix(tokens_map: dict) -> (pd.DataFrame, pd.DataFrame):
    names = sorted(tokens_map.keys())
    sets = {n: set(tokens_map[n]) for n in names}
    n = len(names)
    J = np.zeros((n, n), dtype=np.float32)
    I = np.zeros((n, n), dtype=np.int32)
    for i, a in enumerate(names):
        A = sets[a]
        for j, b in enumerate(names):
            B = sets[b]
            inter = len(A & B)
            union = len(A | B) if (A or B) else 1
            I[i, j] = inter
            J[i, j] = 100.0 * inter / union
    Jdf = pd.DataFrame(J, index=names, columns=names)
    Idf = pd.DataFrame(I, index=names, columns=names)
    return Jdf, Idf

J_all, I_all       = jaccard_matrix(tokens_all_map)
J_nostop, I_nostop = jaccard_matrix(tokens_nostop_map)

J_all.to_csv(OUT_DIR / "jaccard_with_stopwords.csv")
I_all.to_csv(OUT_DIR / "intersection_with_stopwords.csv")
J_nostop.to_csv(OUT_DIR / "jaccard_without_stopwords.csv")
I_nostop.to_csv(OUT_DIR / "intersection_without_stopwords.csv")

J_all.shape, J_nostop.shape


## 9) Correlation of Term‑Frequency Vectors (Pearson)

In [ ]:

corr_all    = dtm_all.T.corr(method="pearson")
corr_nostop = dtm_nostop.T.corr(method="pearson")

corr_all.to_csv(OUT_DIR / "correlation_with_stopwords.csv")
corr_nostop.to_csv(OUT_DIR / "correlation_without_stopwords.csv")

corr_all.iloc[:5, :5]


## 10) Heatmaps (Matplotlib)

In [ ]:

import matplotlib.pyplot as plt

def plot_heatmap(df: pd.DataFrame, title: str, fname: str, vmin=None, vmax=None):
    plt.figure(figsize=(max(8, len(df)/2), max(6, len(df)/2)))
    plt.imshow(df.values, aspect='auto', vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.colorbar()
    plt.xticks(range(len(df.columns)), df.columns, rotation=90, fontsize=8)
    plt.yticks(range(len(df.index)), df.index, fontsize=8)
    plt.tight_layout()
    out_path = OUT_DIR / fname
    plt.savefig(out_path, dpi=200)
    plt.close()
    print(f"Saved {out_path}")

# Jaccard % heatmaps
plot_heatmap(J_all,    "Jaccard Overlap (%) – WITH Stopwords",    "heatmap_jaccard_with_stopwords.png", vmin=0, vmax=100)
plot_heatmap(J_nostop, "Jaccard Overlap (%) – WITHOUT Stopwords", "heatmap_jaccard_without_stopwords.png", vmin=0, vmax=100)

# Correlation heatmaps (Pearson in [-1, 1])
plot_heatmap(corr_all,    "Pearson Correlation – WITH Stopwords",    "heatmap_corr_with_stopwords.png",    vmin=-1, vmax=1)
plot_heatmap(corr_nostop, "Pearson Correlation – WITHOUT Stopwords", "heatmap_corr_without_stopwords.png", vmin=-1, vmax=1)

print("All heatmaps saved to outputs/.")


## 11) Intra‑book & Inter‑book Analysis

In [ ]:

def subset_by_subject(df: pd.DataFrame, subject: str) -> pd.DataFrame:
    rows = [i for i in df.index if i.startswith(subject + "_")]
    cols = [c for c in df.columns if c.startswith(subject + "_")]
    if rows and cols:
        return df.loc[rows, cols]
    return pd.DataFrame()

def cross_subject(df: pd.DataFrame, subj_a: str, subj_b: str) -> pd.DataFrame:
    rows = [i for i in df.index if i.startswith(subj_a + "_")]
    cols = [c for c in df.columns if c.startswith(subj_b + "_")]
    if rows and cols:
        return df.loc[rows, cols]
    return pd.DataFrame()

for subj in SUBJECTS:
    j_sub = subset_by_subject(J_nostop, subj)
    c_sub = subset_by_subject(corr_nostop, subj)
    if not j_sub.empty:
        plot_heatmap(j_sub, f"{subj.title()} Intra‑book Jaccard (%) – No Stopwords",
                     f"heatmap_jaccard_without_stopwords__{subj}.png", vmin=0, vmax=100)
    if not c_sub.empty:
        plot_heatmap(c_sub, f"{subj.title()} Intra‑book Correlation – No Stopwords",
                     f"heatmap_corr_without_stopwords__{subj}.png", vmin=-1, vmax=1)

for i in range(len(SUBJECTS)):
    for j in range(i+1, len(SUBJECTS)):
        a, b = SUBJECTS[i], SUBJECTS[j]
        j_cross = cross_subject(J_nostop, a, b)
        c_cross = cross_subject(corr_nostop, a, b)
        if not j_cross.empty:
            plot_heatmap(j_cross, f"Inter‑book Jaccard (%) {a}→{b} – No Stopwords",
                         f"heatmap_jaccard_without_stopwords__{a}_to_{b}.png", vmin=0, vmax=100)
        if not c_cross.empty:
            plot_heatmap(c_cross, f"Inter‑book Correlation {a}→{b} – No Stopwords",
                         f"heatmap_corr_without_stopwords__{a}_to_{b}.png", vmin=-1, vmax=1)


## 12) One‑Click Runner

In [ ]:

print("✅ Pipeline ready. Run cells in order (or Runtime → Run all).")
print("Artifacts saved in:", OUT_DIR.resolve())



## 13) Upgrades (to go beyond expectations)
- **POS filters**: restrict vocab to NOUN/PROPN/VERB/ADJ for sharper topics.
- **TF–IDF weighting**: replace raw counts (e.g., scikit‑learn `TfidfVectorizer`).
- **n‑grams**: include bigrams/trigrams (*cell division*, *quantum mechanics*).
- **Clustering**: K‑Means/hierarchical clustering + dendrograms.
- **Embeddings**: Sentence‑Transformers for semantic similarity heatmaps.
- **Caching**: Save token lists/DTMs to Parquet for 3000+ pages performance.
